In [ ]:
from stand_da import compute_p_value, generate_data, AD_DA
from model import WDGRL, AutoEncoder
import torch
import numpy as np

In [ ]:
ns, nt, d = 50, 25, 10
alpha = 0.05

mu_s = 0
mu_t = 2
delta_s = [0, 0.5, 1, 1.5, 2]
delta_t = [0] # No true outliers


Xs, ys, sigma_s = generate_data(mu_s, delta_s, ns, d)
Xt, yt, sigma_t = generate_data(mu_t, delta_t, nt, d)


In [4]:
generator_hidden_dims = [50, 10]
critic_hidden_dims = [10]
wdgrl = WDGRL(input_dim=d, generator_hidden_dims=generator_hidden_dims, critic_hidden_dims=critic_hidden_dims)
index = None

with open("model/wdgrl_models.txt", "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        words = line[:-1].split("/")
        if words[1] == str(generator_hidden_dims) and words[2] == str(critic_hidden_dims):
            index = i
            break
        
if index is None:
    print("Error: WDGRL Model not found wdgrl")

check_point = torch.load(f"model/wdgrl_{index}.pth", map_location=wdgrl.device, weights_only=True)
wdgrl.generator.load_state_dict(check_point['generator_state_dict'])
wdgrl.critic.load_state_dict(check_point['critic_state_dict'])
wdgrl.generator = wdgrl.generator.double()


In [5]:
input_dim = generator_hidden_dims[-1]
encoder_hidden_dims = [input_dim, 8, 4, 2]
decoder_hidden_dims = [2, 4, 8, input_dim]
ae = AutoEncoder(input_dim=input_dim, encoder_hidden_dims=encoder_hidden_dims, decoder_hidden_dims=decoder_hidden_dims)
index = None

with open("model/ae_models.txt", "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        words = line[:-1].split("/")
        if words[1] == str(input_dim) and words[2] == str(encoder_hidden_dims) and words[3] == str(decoder_hidden_dims):
            index = i
            break

if index is None:
    print("Error: AE model not found")
check_point = torch.load(f"model/ae_{index}.pth", map_location=ae.device, weights_only=True)
ae.load_state_dict(check_point['state_dict'])
ae = ae.double()

In [ ]:
O = AD_DA(Xs, Xt, wdgrl, ae, alpha=alpha)

j = np.random.choice(O)
p_value = compute_p_value(Xs, Xt, sigma_s, sigma_t, wdgrl, ae, j, O, alpha=alpha)

print(f"Set of anomalies: {O}")
print(f"p-value for anomaly {j}: {p_value}")

Set of anomalies: [ 0 14 17]
p-value for anomaly 14: 0.9289881247100045
